In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
function get_boundary_filtration(points, n_atoms_per_mol)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_boundary_filtration(points, n_atoms_per_mol):
        points = np.asarray(points)

        simplices = diode.fill_alpha_shapes(points)

        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
        
        def is_multi(sigma):
            has_a = has_b = False
            for v in sigma.vertices:
                if v < n_atoms_per_mol:
                    has_a = True
                else:
                    has_b = True
            return has_a and has_b
        
        fil = fil.subfiltration(is_multi)

        return fil
    """
    py"get_boundary_filtration"(points, n_atoms_per_mol)
end

get_boundary_filtration (generic function with 1 method)

In [4]:
function get_multichromatic_edges_of_tetrahedron(verts; split = 1206)
    a,b,c,d = verts
    combinations = [[a,b], [a,c], [a,d], [b,c], [b,d], [c,d]]
    [e for e in combinations if any([v > split for v in e]) && any([v <= split for v in e])]
end

function get_barycenters_of_multichromatic_edges(edges, points)
    barycenters = Vector{Point3f}([])
    for e in edges
        a,b = e[1],e[2]
        bc = Point3f(0.5 * (points[a] + points[b]))
        push!(barycenters, bc)
    end
    if length(barycenters) == 4
        bc = Point3f(0.25 * (barycenters[1] + barycenters[2] + barycenters[3] + barycenters[4]))
        push!(barycenters, bc)
    end 
    barycenters
end

function get_faces_from_barycenters(barycenters)
    if length(barycenters) == 3
        faces = [1 2 3]
    elseif length(barycenters) == 5
        faces = [1 2 5; 3 4 5; 2 4 5; 1 3 5]
        #3 4 5; 1 4 5
    end
    faces
end

get_faces_from_barycenters (generic function with 1 method)

In [17]:
#@load "assets/output/persistence/exploding.jld2" input output
@load "../../Data/collected_simulation_results/rwm_wp_2_6r7m/43.jld2"

points = MorphoMol.Utilities.get_matrix_realization(output["states"][1], input["template_centers"])
points = [e for e in eachcol(hcat(points...))];
n_atoms_per_mol = length(input["template_radii"])
n_mol = Int(length(points) / 1206)

2

In [18]:
fil = get_boundary_filtration(points, 1206);

In [36]:
for cell in fil.cells()
    parts = [Vector{Int}([]) for i in 1:n_mol]
    for v in cell.vertices
        push!(parts[div(v, n_atoms_per_mol) + 1], v)
    end
    cell.value = 0.0
end

In [8]:
min_z = minimum([e[3] for e in points])
max_z = maximum([e[3] for e in points])

55.705481108505744

In [39]:
f = Figure(fontsize = 7)
lscene = LScene(f[1, 1])

#mesh_ax = Axis3(f[1,1])

for c in fil.cells()
    if length(c.vertices) == 4
        mce = get_multichromatic_edges_of_tetrahedron(c.vertices)
        bcs = get_barycenters_of_multichromatic_edges(mce, points)
        faces = get_faces_from_barycenters(bcs)
        colors = [(b[3] - min_z) / (max_z - min_z) for b in bcs]
        mesh!(lscene, bcs, faces, color = colors, colorrange = (0,1))
    end
end

display(f)

GLMakie.Screen(...)

In [14]:
f = Figure(fontsize = 7)
conf_ax = Axis3(f[1,1])

c1_points = [Point3f(p) for p in points[1:1206]]
c2_points = [Point3f(p) for p in points[1207:end]]

conf_ms = 10
scatter!(conf_ax, c1_points, markersize = conf_ms)
scatter!(conf_ax, c2_points, markersize = conf_ms)

for c in fil.cells()
    if length(c.vertices) == 3
        i = c.vertices[1]
        j = c.vertices[2]
        k = c.vertices[3]
        a = Point3f(points[i])
        b = Point3f(points[j])
        c = Point3f(points[k])
        c1 = i <= 1206 ? 1 : 2
        c2 = j <= 1206 ? 1 : 2
        c3 = k <= 1206 ? 1 : 2

        lines!(conf_ax, [a,b], color = [c1, c2])
        lines!(conf_ax, [a,c], color = [c1, c3])
        lines!(conf_ax, [a,b], color = [c2, c3])
    end
end

mesh_ax = Axis3(f[1,2])

for c in fil.cells()
    if length(c.vertices) == 4
        h = c.vertices[1]
        i = c.vertices[2]
        j = c.vertices[3]
        k = c.vertices[4]

        mce = get_multichromatic_edges_of_tetrahedron(c.vertices)
        bcs = get_barycenters_of_multichromatic_edges(mce, points)
        faces = get_faces_from_barycenters(bcs)

        mesh!(bcs, faces)
    end
end

display(f)

GLMakie.Screen(...)